Web Scraping Boot_Camp_04/01/2023 || Lochan Basyal

In [76]:
import requests
sitemap_url = 'https://www.rottentomatoes.com/sitemaps/sitemap.xml'
sitemap = requests.get(sitemap_url).text

In [77]:
import xml.etree.ElementTree as ET
tree = ET.ElementTree(ET.fromstring(sitemap))
print(tree)

In [78]:
from pprint import pprint
parsed_tree = list(tree.getroot().iter())
pprint(parsed_tree[:5])
len(parsed_tree)

[<Element '{http://www.sitemaps.org/schemas/sitemap/0.9}sitemapindex' at 0x7f82fd5d0950>,
 <Element '{http://www.sitemaps.org/schemas/sitemap/0.9}sitemap' at 0x7f82fd5d07c0>,
 <Element '{http://www.sitemaps.org/schemas/sitemap/0.9}loc' at 0x7f82fd5d0310>,
 <Element '{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod' at 0x7f82fd5d08b0>,
 <Element '{http://www.sitemaps.org/schemas/sitemap/0.9}sitemap' at 0x7f82fd5d0590>]


355

In [79]:
# Make an empty list to contain output
movie_sitemap_urls = list()
# Loop through parsed_tree
for element in parsed_tree:
    ele_text = element.text  # Get text from child element
    # Look for urls containing 'movie'
    if 'movie' in ele_text:
        movie_sitemap_urls.append(ele_text)  # Append each url to output
    continue
# Examine output
pprint(movie_sitemap_urls[:5])
len(movie_sitemap_urls)

['https://www.rottentomatoes.com/sitemaps/movie_0.xml',
 'https://www.rottentomatoes.com/sitemaps/movie_1.xml',
 'https://www.rottentomatoes.com/sitemaps/movie_2.xml',
 'https://www.rottentomatoes.com/sitemaps/movie_3.xml',
 'https://www.rottentomatoes.com/sitemaps/movie_4.xml']


18

In [80]:
print(f'Example: {movie_sitemap_urls[0]}')
movie_urls = list()  # Empty list to store resuslts
response = requests.get(movie_sitemap_urls[0])  # Use request to get content
tree = ET.ElementTree(ET.fromstring(response.text))  # Get XML Tree
parsed_tree = list(tree.getroot().iter())  # Make tree iterable
for url in parsed_tree:
    # for each 'url' element, there's 2 other elements: 'loc' and 'image'
    # we need to loop through them and find each 'loc' for urls
    for element in url:
        if 'loc' in element.tag and 'rottentomatoes.com/m/' in element.text:
            movie_urls.append(element.text)
        continue
# print(url)
# Examine Results
print('Results: ', end='')
pprint(movie_urls[:5])
len(movie_urls)

Example: https://www.rottentomatoes.com/sitemaps/movie_0.xml
Results: ['https://www.rottentomatoes.com/m/limo',
 'https://www.rottentomatoes.com/m/limo/pictures',
 'https://www.rottentomatoes.com/m/beg1994',
 'https://www.rottentomatoes.com/m/beg1994/pictures',
 'https://www.rottentomatoes.com/m/tejano']


17425

In [81]:
# Empty list to store all results
all_movie_urls = list()
for url in movie_sitemap_urls:
    response = requests.get(url)
    tree = ET.ElementTree(ET.fromstring(response.text))
    parsed_tree = list(tree.getroot().iter())
    for url in parsed_tree:
        for element in url:
            if 'loc' in element.tag and 'rottentomatoes.com/m/' in element.text:
                all_movie_urls.append(element.text)
            continue
        continue
    continue
# De-duplicate
all_movie_urls = list(set(all_movie_urls))
# Examine Results
print(f'{len(all_movie_urls)} Total URLs: ', end='')
pprint(all_movie_urls[:5])

251256 Total URLs: ['https://www.rottentomatoes.com/m/legacy_of_rage',
 'https://www.rottentomatoes.com/m/house_of_sin',
 'https://www.rottentomatoes.com/m/revolutionary_road/pictures',
 'https://www.rottentomatoes.com/m/city_of_angels/pictures',
 'https://www.rottentomatoes.com/m/still_the_enemy_within']


Scrap using beautiful soup

In [82]:
from bs4 import BeautifulSoup
from pprint import pprint
from time import sleep
import requests
import re

In [83]:
example_movie_url = 'https://www.rottentomatoes.com/m/top_gun_maverick'
# Make soup
soup = BeautifulSoup(requests.get(example_movie_url).content,
'html.parser')
# print(soup)

In [84]:
# Create Empty Dictionary to hold all data
output = dict()
# Find parent element
target_class = 'panel-body content_body'
p_block = soup.find(class_=target_class)
# pprint(p_block)
# Find Synopsis
movie_synopsis = p_block.find(slot='content')
movie_synopsis = movie_synopsis.text.strip()
print(movie_synopsis)
# Insert result into dictionary
output['Movie Synopsis'] = movie_synopsis

After more than thirty years of service as one of the Navy’s top aviators, Pete “Maverick” Mitchell (Tom Cruise) is where he belongs, pushing the envelope as a courageous test pilot and dodging the advancement in rank that would ground him. When he finds himself training a detachment of Top Gun graduates for a specialized mission the likes of which no living pilot has ever seen, Maverick encounters Lt. Bradley Bradshaw (Miles Teller), call sign: “Rooster,” the son of Maverick’s late friend and Radar Intercept Officer Lt. Nick Bradshaw, aka “Goose”.

Facing an uncertain future and confronting the ghosts of his past, Maverick is drawn into a confrontation with his own deepest fears, culminating in a mission that demands the ultimate sacrifice from those who will be chosen to fly it.


In [85]:
# Find All Other Info: Genre/Language/etc.
# Using find_all() to get all rows, then parse detail info from the root element
meta_values = p_block.find_all(class_='info-item')
print(meta_values)
# Inside each row, find its 'label' and 'value'
for mv in meta_values:
    label = mv.find(class_='info-item-label').text.strip()
    value = mv.find(class_='info-item-value').text.strip()
    output[label] = value  # Insert into output dictionary
    print(f'{label}: {value}')
    continue
# Clean fields
for key in output:
    original_value = output[key]
    new_value = original_value.replace('\n', '')  # Remove all new-lines
    new_value = re.sub(' +', ' ', new_value)  # Remove consequtive white- spaces
    output[key] = new_value  # Re-assign into the output dictionary
    continue
# Examine Result
pprint(output)

[<li class="info-item" data-qa="movie-info-item">
<p>
<b class="info-item-label" data-qa="movie-info-item-label">Rating:</b>
<span class="info-item-value" data-qa="movie-info-item-value">PG-13 (Some Strong Language|Sequences of Intense Action)
                        </span>
</p>
</li>, <li class="info-item" data-qa="movie-info-item">
<p>
<b class="info-item-label" data-qa="movie-info-item-label">Genre:</b>
<span class="info-item-value" data-qa="movie-info-item-value" genre"="">
                            
                            Action, 
                            
                            Adventure
                            
                        </span>
</p>
</li>, <li class="info-item" data-qa="movie-info-item">
<p>
<b class="info-item-label" data-qa="movie-info-item-label">Original Language:</b>
<span "="" class="info-item-value" data-qa="movie-info-item-value">English</span>
</p>
</li>, <li class="info-item" data-qa="movie-info-item">
<p>
<b class="info-item-label" d

In [86]:
# Create empty dictionary to hold them all
all_movies = dict()
for url in all_movie_urls[:10]:
    output = dict()
    # Step 1: Make soup
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    # Step 2: Find parent element
    target_class = 'panel-body content_body'
    p_block = soup.find(class_=target_class)
    # Step 3: Find child elements
    if not p_block:
        continue  # skip to next movie
    try:
        movie_synopsis = p_block.find(slot='content')
        movie_synopsis = movie_synopsis.text.strip()
        output['Movie Synopsis'] = movie_synopsis
    except AttributeError:
        output['Movie Synopsis'] = None
        pass
    # Find individual meta-values:
    # Using find_all() to get each row
    meta_values = p_block.find_all(class_='info-item')
    # Inside each row, find its 'label' and 'value'
    for mv in meta_values:
        label = mv.find(class_='info-item-label').text.strip()
        value = mv.find(class_='info-item-value').text.strip()
        output[label] = value
        continue
    # Clean fields
    for key in output:
        original_value = output[key]
        new_value = original_value.replace('\n', '')
        new_value = re.sub(' +', ' ', new_value)
        output[key] = new_value
        continue
    all_movies[url] = output
    pprint(output)
    # sleep(1)
    continue

{'Director:': 'Ronnie Yu',
 'Genre:': 'Action',
 'Movie Synopsis': 'Sentenced to prison for a crime he did not commit, a man '
                   'sets out for revenge after his release from jail.',
 'Original Language:': 'Chinese',
 'Rating:': 'R',
 'Release Date (Streaming):': 'Apr 5, 2005',
 'Runtime:': '1h 26m'}
{'Director:': 'Philip Gardiner, Richard King, Nik Spencer, John Symes',
 'Genre:': 'Drama',
 'Movie Synopsis': 'A London hotel houses guests that are free to indulge in '
                   'their perverse desires.',
 'Original Language:': 'English',
 'Release Date (Streaming):': 'Mar 13, 2017',
 'Runtime:': '1h 20m'}
{'Director:': 'Owen Gower',
 'Genre:': 'Documentary, History, Drama',
 'Movie Synopsis': "Miners who lived through the longest strike in Britain's "
                   'history recount the yearlong events 30 years later.',
 'Original Language:': 'English (United Kingdom)',
 'Producer:': 'Sinead Kirwan, Mark Lacey',
 'Runtime:': '1h 52m',
 'Writer:': 'Owen Gowe

In [88]:
import pandas as pd
pd.DataFrame(all_movies).T

,Movie Synopsis,Rating:,Genre:,Original Language:,Director:,Release Date (Streaming):,Runtime:,Producer:,Writer:,Sound Mix:,Release Date (Theaters):,Distributor:
https://www.rottentomatoes.com/m/legacy_of_rage,Sentenced to prison for a crime he did not com...,R,Action,Chinese,Ronnie Yu,"Apr 5, 2005",1h 26m,NaN,NaN,NaN,NaN,NaN
https://www.rottentomatoes.com/m/house_of_sin,A London hotel houses guests that are free to ...,NaN,Drama,English,"Philip Gardiner, Richard King, Nik Spencer, Jo...","Mar 13, 2017",1h 20m,NaN,NaN,NaN,NaN,NaN
https://www.rottentomatoes.com/m/still_the_enemy_within,Miners who lived through the longest strike in...,NaN,"Documentary, History, Drama",English (United Kingdom),Owen Gower,NaN,1h 52m,"Sinead Kirwan, Mark Lacey",Owen Gower,NaN,NaN,NaN
https://www.rottentomatoes.com/m/out_of_season,After running a seaside roller coaster over th...,R (Language|Strong Violence|Some Sexuality),"Mystery & thriller, Crime, Drama",English,Jevon O'Neill,"Oct 4, 2005",1h 40m,Kim Leggatt,Jevon O'Neill,Dolby Digital,NaN,NaN
https://www.rottentomatoes.com/m/desert_one,Filmmaker Barbara Kopple examines a failed U.S...,NaN,Documentary,English,Barbara Kopple,"Aug 21, 2020",1h 47m,NaN,NaN,NaN,"Aug 21, 2020 limited",Greenwich Entertainment
https://www.rottentomatoes.com/m/do_it_in_post,Alex must find out if he has what it takes to ...,NaN,Comedy,English,Rob Dionne,"Sep 9, 2016",1h 22m,"Tremain Brown, Rob Dionne, Andrew Schwartz",Rob Dionne,NaN,NaN,NaN


Scrape using selenium

In [89]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType

In [90]:
options = Options()
#options.headless = True
# options.add_argument('log-level=3')
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
# options.add_argument('--disable-dev-shm-usage')  # overcome limited resource problems
# options.add_argument('--no-sandbox')  # Bypass OS security model
# options.add_argument('--window-size=1280,710')
# options.add_argument('--ignore-certificate-errors-spki-list')
driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install(), chrome_options=options)

/var/folders/2m/tvvx0pgs3d9gqpkmnm0y179h0000gp/T/ipykernel_24744/113082776.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install(), chrome_options=options)
/var/folders/2m/tvvx0pgs3d9gqpkmnm0y179h0000gp/T/ipykernel_24744/113082776.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install(), chrome_options=options)


In [91]:
# open the webpage
driver.get('https://www.rottentomatoes.com/browse/movies_in_theaters')

In [93]:
while True:
    try:
        # scroll to the bottom of the page
        _ = driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        # locate the Load More Button
        load_more_button = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='discovery__actions']/button")))
        # Click the button
        load_more_button.click()
    except:
        break
# locate all the movie elements
movies_elements = driver.find_elements(by = By.XPATH, value = "//a[@class='js-tile-link ']")
len(movies_elements)

33

In [94]:
# use list comprehension instead
movies_list = [x.get_attribute('href') for x in movies_elements]
len(movies_list)
print(movies_list[0])

https://www.rottentomatoes.com/m/santiago_the_camino_within


In [50]:
# getting all the movie info from each movie
smaller_list = movies_list[:5]    # creating a smaller subset of list to test out the code
row_list = list()   # initiating a row list where each element is one row of our future df
for each_movie in smaller_list:
    # open the movie page
    driver.get(each_movie)
    # locate and extract the movie synopsis text blob
    movieSynopsis = driver.find_element(by=By.XPATH, value="//p[@data-qa='movie-info-synopsis']").text
    # grab the movieinfo box
    movieInfo = driver.find_elements(by=By.XPATH,value="//ul[@id='info']/li")
    row_dict = dict()   # initiating a row dict to store all the column-value pairs of each row
    for each_info in movieInfo:
        field = each_info.text
        k, v = field.split(': ')
        row_dict[k] = v
    row_dict['movie_id'] = each_movie
    row_dict['synopsis'] = movieSynopsis
    row_list.append(row_dict)
# create a pandas df from the scraped data
import pandas as pd
df = pd.DataFrame(row_list)
df.head(3).T

,0,1,2
Genre,Documentary,"Action, Adventure, Drama","Documentary, Biography, Sports"
Original Language,English,Telugu,English
Director,Unknown Director,Srikanth Odhela,Dylan Mulick
Release Date (Theaters),"Mar 28, 2023 Limited","Mar 29, 2023 Limited","Mar 30, 2023 Limited"
Runtime,1h 45m,2h 0m,1h 33m
Distributor,Fathom Events,Friday Entertainment,Fathom Events
movie_id,https://www.rottentomatoes.com/m/santiago_the_...,https://www.rottentomatoes.com/m/dasara,https://www.rottentomatoes.com/m/helen_believe
synopsis,The Camino de Santiago is a well-traversed pil...,,"From Producer Chris Pratt, Helen: Believe foll..."
Producer,NaN,Sudhakar Cherukuri,"Tracy Aftergood, Andrea Courtney, Sean Deverea..."
Writer,NaN,Srikanth Odhela,Dylan Mulick
